In [1]:
import os
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model, Model, model_from_json
import cv2
import tkinter as tk
import numpy as np
import tensorflow as tf


In [2]:
from dress_color import estimate_dress_colour, nearest_colour_name, dominant_colour_kmean
from age_gender_prediction import detect_faces, predict_age, predict_gender,face_net,face_proto,age_net,gender_net

In [3]:
emotion_model = load_model("emotion_detector.h5")

with open('model_arch.json') as json_file:
    loaded_model= json_file.read()

emotion_model = model_from_json(loaded_model)

emotion_model.load_weights("model.weights.h5")

emotion_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print("Model loaded and compiled successfull")

Model loaded and compiled successfull


C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 20 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
nationality_model= load_model("nationality_detection.h5")

with open('nationality_arch.json') as json_file:
    loaded_model= json_file.read()

nationality_model = model_from_json(loaded_model)

nationality_model.load_weights("nationalitymodel.weights.h5")

nationality_model.compile(loss = "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print("Model loaded and compiled successfully")

Model loaded and compiled successfully


In [5]:
EMO_INPUT_SIZE = 48
EMOTION_LABELS = ["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"]

NAT_INPUT_SIZE = 224
NAT_LABELS = ["Indian","USA","African","Other"]

In [6]:
def predict_emotion(face_bgr):
    gray = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(gray, (EMO_INPUT_SIZE, EMO_INPUT_SIZE))
    img = img.astype("float32")/255.0
    img= np.expand_dims(img, axis= -1)
    img = np.expand_dims(img , axis= 0)
    preds = emotion_model.predict(img, verbose=[0])[0]
    idx = int(np.argmax(preds))
    return EMOTION_LABELS[idx], float (preds[idx]), preds

In [7]:
def predict_nationality(face_bgr):
    img = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (NAT_INPUT_SIZE,NAT_INPUT_SIZE))
    img = img.astype("float32")/255.0
    img = np.expand_dims(img, 0)
    preds = nationality_model.predict(img, verbose= 0)[0]
    idx = int(np.argmax(preds))
    return NAT_LABELS[idx], float (preds[idx]), preds


In [ ]:
class App:
    def __init__ (self,root):
        self.root = root
        root.title("Nationality , Gender, Age, Dress Color and Emotion detection GUI" )
        root.geometry("1000x640")
        self.img_bgr = None
        self.imgtk = None
        self.canvas = tk.Canvas(root, width = 600, height= 520, bg= "black")
        self.canvas.place(x=20, y=20)

        tk.Button(root, text="Open Image", command=self.open_image).place(x=600, y=30, width=120,  height=40)
        tk.Button(root, text= "Run", command=self.run_inference).place(x=700, y=30, width=120, height=40)
        tk.Button(root, text="Clear", command= self.clear).place(x=820, y=30, width= 60, height= 40)
        self.output= tk.Text(root, width= 40, height= 28)
        self.output.place(x= 640, y=80)

    
    def open_image(self):
        path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp"), ("All files", "*.*")])

        if not path:
            return
        img = cv2.imread(path)
        if img is None:
            messagebox.showerror("Error", "Failed to load Image")
            return
        self.img_bgr = img
        self.show_preview(img)
    
    def show_preview(self, img_bgr, boxes = None):
        img = img_bgr.copy()
        if boxes:
            for (x1, y1, x2, y2, conf) in boxes:
                cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        h,w = img.shape[:2]
        scale = min(600/w, 520/h)
        img = cv2.resize(img, (int(w*scale), int(h*scale)))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil = Image.fromarray(img_rgb)
        self.imgtk = ImageTk.PhotoImage(pil)
        self.canvas.delete("all")
        self.canvas.create_image(300,260, image= self.imgtk)
        
    def run_inference(self):
        if self.img_bgr is None:
            messagebox.showinfo("Info", "Open an image first")
            return
        frame = self.img_bgr.copy()
        boxes = detect_faces(face_net, frame, conf_threshold=0.6)
        if not boxes or len(boxes[0]) < 4:
            messagebox.showinfo("No face", "No face detected")
            return
        
        normalized_boxes = []

    
        for b in boxes:
            try:
                vals = []
                for v in b[:5]:
                    if isinstance(v, (list, tuple, np.ndarray)):
                        arr = np.array(v).flatten()
                        vals.append(float(arr[0]))
                    else:
                        vals.append(float(v))

                if len(vals) >= 4:
                    if len(vals) == 5:
                        x1, y1, x2, y2, conf = vals
                    else:
                        x1, y1, x2, y2 = vals[:4]
                        conf = 1.0

    
                    if x2 < 100 and y2 < 100:
                        x2 = x1 + x2
                        y2 = y1 + y2

                    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                    normalized_boxes.append((x1, y1, x2, y2, conf))

            except Exception as e:
                print("⚠️ Skipping box due to error:", e, " | Raw box:", b)



        if not normalized_boxes:
            messagebox.showinfo("No face", "No valid face detected")
            return

    
        boxes = sorted(normalized_boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)
        x1, y1, x2, y2, conf = boxes[0]
        face = frame[y1:y2, x1:x2].copy()

    
        if face.size == 0:
            messagebox.showerror("Face crop error", "Face crop empty")
            return

    
        nat_label, nat_conf, _ = predict_nationality(face)
        emo_label, emo_conf, _ = predict_emotion(face)
        age_pred = predict_age(face,age_net)
        gender_pred= predict_gender(face,gender_net)
        dress_colour = estimate_dress_colour(frame, (x1,y1,x2,y2))

    
        text_lines = []
        text_lines.append(f"Nationality: {nat_label}  ({nat_conf*100:.1f}%)")
        text_lines.append(f"Emotion:     {emo_label}  ({emo_conf*100:.1f}%)")
        text_lines.append(f"Gender:      {gender_pred}")
        if nat_label == "Indian":
            text_lines.append(f"Age:         {age_pred}")
            text_lines.append(f"Dress color: {dress_colour}")
        elif nat_label == "USA":
            text_lines.append(f"Age:         {age_pred}")
        elif nat_label == "African":
            text_lines.append(f"Dress color: {dress_colour}")
    
        self.output.delete("1.0", tk.END)
        self.output.insert(tk.END, "\n".join(text_lines))

    
        self.show_preview(self.img_bgr, boxes=[boxes[0]])

    def clear(self):
        self.img_bgr = None
        self.canvas.delete("all")
        self.output.delete("1.0", tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()